### Magics

In [1]:
%load_ext autoreload
%autoreload 2

### Imports

In [2]:
#! pip3 install munch

In [2]:
import sounddevice as sd
import soundfile as sf
import time
import IPython
import time
from collections import deque

# Serialization for device config files
from munch import munchify
import yaml

# Package imports
from midi_controller import MIDIController


### Variables & Setup

In [3]:
# Input name of device
device_name = "Boss SD-1w"

# Select the type of controller
controller_type = ["MIDI", "arduino"][0]

midi_sleep_dur = 2
# install mido / python-rtmidi
#data_dir = "out/"
data_dir = "./data/" #"/Volumes/16GB THUMBD/data/chase bliss mkII/"
input_file = '/Users/narad/Desktop/projects/tone-render/di/real/lorcan/003_lorcan_metal_rhythm.wav' 
#'/Users/narad/Desktop/projects/amp-space/data/scripts/python/test_di.wav'
start_second = 3
duration = 6

### Create Controller

In [4]:
if controller_type == "MIDI":
    controller = MIDIController()
else: # Arduino
    controller = ServoController()

['Clarett 4Pre USB', 'Clarett 4Pre USB']
['Clarett 4Pre USB', 'Clarett 4Pre USB']


### Setup Recording Device

In [5]:
# Setup Audio Recording loop

device_info = sd.query_devices(0, 'input')
print(device_info)

samplerate = int(device_info['default_samplerate'])

sd.default.samplerate = samplerate
sd.default.channels = 2
#devices = sd.query_devices()
print(device_info)

{'name': 'Clarett 4Pre USB', 'index': 0, 'hostapi': 0, 'max_input_channels': 18, 'max_output_channels': 8, 'default_low_input_latency': 0.01, 'default_low_output_latency': 0.004354166666666667, 'default_high_input_latency': 0.1, 'default_high_output_latency': 0.0136875, 'default_samplerate': 48000.0}
{'name': 'Clarett 4Pre USB', 'index': 0, 'hostapi': 0, 'max_input_channels': 18, 'max_output_channels': 8, 'default_low_input_latency': 0.01, 'default_low_output_latency': 0.004354166666666667, 'default_high_input_latency': 0.1, 'default_high_output_latency': 0.0136875, 'default_samplerate': 48000.0}


### Load Data

In [18]:
# Read input file
data, sr = sf.read(input_file, dtype='float32', samplerate=None)
print("Sample rate of DI track: ", sr)
data = data[start_second * sr:(start_second + duration) * sr]

# Display to test
sf.write(f"{data_dir}di.wav", data, sr)
IPython.display.Audio(data, rate=sr)


Sample rate of DI track:  48000


In [22]:
# Read config file for device & setup sweeps

with open("config/axe_fx_midi_controls.yaml", "r") as stream:
    try:
        device_dict = munchify(yaml.safe_load(stream))
#        print(device_dict)
    except yaml.YAMLError as exc:
        print(exc)

for i, knob in enumerate(device_dict.parameters):
    print(i, ": ", knob.name)
    if knob.parameter_type == "discrete":
#        print(knob.values)
        print(knob)
        print(knob.midi)
        for v in knob.settings:
            print(v.name)
    print()
    

0 :  Treble

1 :  Mids

2 :  Bass

3 :  Gain



In [31]:
from sweeps import Sweeper, SweepConfig # import DeviceSettingsSweep

# dsweep = DeviceSettingsSweep(device_dict.parameters)
# print(dsweep.full_sweep())
config = SweepConfig("config/axe_fx_midi_controls.yaml")
sweeper = Sweeper(config, max_samples=1000)

Reducing the number of sweeps in 5150 Block,
  10000 -> 1000


In [52]:
sweeper.write(f"{data_dir}settings.yaml", di_file=input_file)

In [42]:

percent_midi = 126 // 10
#param_sweep = get_settings(sweeps)
#param_sweep.reverse()
#print(len(param_sweep), " sweeps...")

print("calculating param sweep")
#param_sweep = dsweep.get_settings()
#print(len(param_sweep))

control_mapping = {
    'Drive': 11,
    'Bass': 12,
    'Mid': 13,
    'Treble': 14,
}

channel=0

last_setting = dict()
for sweep_name, sweep in sweeper.sweeps:
    sweep = list(sweep)

    for setting in sweep:
        print(setting)
        for pname,pvalue in setting.items():
            print(pname)
            param_control_id = control_mapping[pname]
            param_control_value = int((127 / 10) * (10 * pvalue))
            print(param_control_id)
            print(param_control_value)
            controller.change_setting(channel, param_control_id, param_control_value)
            time.sleep(midi_sleep_dur)

#             if k not in last_setting or last_setting[k] != v:
#     #            print(f"changing {k} to {v}")
#                 param_control_id = controls2midi[k]
#                 if k in sliders:
#                     param_control_value = v * percent_midi
#                 else:
#                     param_control_value = v

#         time.sleep(midi_sleep_dur)
#         last_setting = setting

        # Recording
        recording = sd.playrec(data, sr, device=0, channels=1)
        status = sd.wait()
        pstring = '_'.join([f"{k}-{v}" for k,v in setting.items()])
        sf.write(f"{data_dir}rec_{pstring}.wav", recording, sr)



calculating param sweep
{'Drive': 0.5, 'Bass': 0.7, 'Mid': 0.4, 'Treble': 0.9}
Drive
11
63
OUT:  control_change channel=0 control=11 value=63 time=0
Bass
12
88
OUT:  control_change channel=0 control=12 value=88 time=0
Mid
13
50
OUT:  control_change channel=0 control=13 value=50 time=0
Treble
14
114
OUT:  control_change channel=0 control=14 value=114 time=0
{'Drive': 0.7, 'Bass': 0.3, 'Mid': 0.9, 'Treble': 1.0}
Drive
11
88
OUT:  control_change channel=0 control=11 value=88 time=0
Bass
12
38
OUT:  control_change channel=0 control=12 value=38 time=0
Mid
13
114
OUT:  control_change channel=0 control=13 value=114 time=0
Treble
14
127
OUT:  control_change channel=0 control=14 value=127 time=0
{'Drive': 0.6, 'Bass': 0.5, 'Mid': 0.1, 'Treble': 0.8}
Drive
11
76
OUT:  control_change channel=0 control=11 value=76 time=0
Bass
12
63
OUT:  control_change channel=0 control=12 value=63 time=0
Mid
13
12
OUT:  control_change channel=0 control=13 value=12 time=0
Treble
14
101
OUT:  control_change channel

In [ ]:

#outport.send(msglog.popleft()["msg"])

#         msg = mido.Message('note_on', note=note, velocity=velocity)

# percent_midi = 126 // 5
# for i in range(5+1):
#     print(i, ": ", i * percent_midi)

#portobject._port_out.send(



# Chase Bliss MIDI messages
# https://static1.squarespace.com/static/5dce1364138bbd66dabfb03c/t/5ed13db6ef385f43440b0569/1590771126842/Preamp+MKII_MIDI+Manual_Pedal_Chase+Bliss+Audio.pdf

# CC num 14
# CC val 114
# 127 = 100% slider position


# inport = mido.open_input('DigitalKBD MIDI 1')
# outport = mido.open_output('DigitalKBD MIDI 1')

# msglog = deque()
# echo_delay = 2

# while True:
#     while inport.pending():
#         msg = inport.receive()
#         if msg.type != "clock":
#             print msg
#             msglog.append({"msg": msg, "due": time.time() + echo_delay})
#     while len(msglog) > 0 and msglog[0]["due"] <= time.time():
#         outport.send(msglog.popleft()["msg"])
        

#mido.get_output_names()

In [ ]:
#sounddevice.query_devices(device=2, kind=None)


#sd.play(myarray, fs)

In [ ]:
#try:
#sd.play(data, fs, device=2)
#status = sd.wait()
# except:
#     pass




In [ ]:
def callback(indata, outdata, frames, time, status):
    # assert frames == blocksize
    if status.output_underflow:
        print('Output underflow: increase blocksize?', file=sys.stderr)
        raise sd.CallbackAbort
    assert not status
    try:
        data = qOut.get_nowait()
    except queue.Empty:
        print('Buffer is empty: increase buffersize?', file=sys.stderr)
        raise sd.CallbackAbort
    if len(data) < len(outdata):
        outdata[:len(data)] = data
        outdata[len(data):] = b'\x00' * (len(outdata) - len(data))
        raise sd.CallbackStop
    else:
        outdata[:] = data
    offsetInOut = time.inputBufferAdcTime-time.outputBufferDacTime
    # Place incoming data to fileRec queue - is this correct?
    qIn.put(indata[:])

In [ ]:
try:
    with sf.SoundFile('../1kHz_square_wave.wav') as fileStim:
        with sf.SoundFile('../testRecordingPython.wav', mode='x', samplerate=fileStim.samplerate, channels=1, subtype='FLOAT') as fileRec:
            for _ in range(buffersize):
                data = fileStim.buffer_read(blocksize, dtype='float32')
                if not data:
                    break
                qOut.put_nowait(data)  # Pre-fill queue
            stream = sd.RawStream(blocksize=blocksize, channels=1, dtype='float32', callback=callback, finished_callback=event.set)
            with stream:
                timeout = blocksize * buffersize / fileStim.samplerate
                print('timeout:', timeout)
                while data:
                    data = fileStim.buffer_read(blocksize, dtype='float32')
                    qOut.put(data, timeout=timeout)
                    fileRec.buffer_write(qIn.get())        # grab input buffer and write to fileRec - is this the right place?
                event.wait()  # Wait until playback is finished